In [16]:
import hashlib
import random

In [17]:
def mod_exp(base, exp, mod):
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp = exp >> 1
        base = (base * base) % mod
    return result

In [18]:
def mod_inverse(a, m):
    return pow(a, -1, m)

In [19]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

In [20]:
def generate_elgamal_keys(p, g):
    # x = random.randint(2, p - 2)
    x = 296910705850334802942252230016917197119761411529502398856920411716398776665995117253388109347257855432665575504925621092927500886757763228288976190435218181204860246792441747727687404284118803628280292427401702743706695331998886442243964393075281831948230289332261342065707450523522625876551455787909702212448
    y = mod_exp(g, x, p)
    return (p, g, y), x

In [21]:
def encrypt_elgamal(message, public_key):
    p, g, y = public_key
    #Khóa phiên k được random mỗi lần chạy vậy nên bản mã sẽ khác nhau
    #Trong ví dụ trong file word đính kèm, khóa phiên k có giá trị
    
    k = 231829683515004285894248732404818838505780640291715123088889449666416885328634740604061367009552293107004845199650478342025302430924465040754743126387968600172838303296241667234672358030056988574445077392877459316860987264706489823044426733783629485383174221974246710107609888722370369714382438004845162844505
    
    #Hàm tính k ngẫu nhiên
    # k = random.randint(2, p - 2)
    # while gcd(k, p - 1) != 1:
    #     k = random.randint(2, p - 2)
    #Kết thúc hàm tính k ngẫu nhiên
    print(f"k: {k}")
    s = mod_exp(y, k, p)
    c1 = mod_exp(g, k, p)
    message_int = int.from_bytes(message.encode(), 'big')  # Convert message to integer
    c2 = (message_int * s) % p
    
    return c1, c2

In [22]:
def decrypt_elgamal(ciphertext, private_key, p):
    c1, c2 = ciphertext
    x = private_key
    
    s = mod_exp(c1, x, p)
    
    s_inverse = pow(s, -1, p)
    
    message_int = (c2 * s_inverse) % p
    
    message_bytes = message_int.to_bytes((message_int.bit_length() + 7) // 8, 'big')
    return message_bytes.decode()


In [23]:
def is_prime(n, k=128):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


In [24]:
def generate_large_prime(bits=2048):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p):
            return p

In [25]:
def find_primitive_root(p):
    if p == 2:
        return 1  

    phi = p - 1
    prime_factors = find_prime_factors(phi)
    
    for g in range(2, min(p, 1000)):  
        is_primitive = True
        for q in prime_factors:
            if pow(g, phi // q, p) == 1:
                is_primitive = False
                break
        if is_primitive:
            return g  
    
    return None  

In [26]:
def find_prime_factors(n):
    factors = set()
    while n % 2 == 0:
        factors.add(2)
        n //= 2
    for i in range(3, int(n**0.5) + 1, 2):
        while n % i == 0:
            factors.add(i)
            n //= i
    if n > 2:
        factors.add(n)
    return factors

In [27]:
def hash_message(message):
    hash_object = hashlib.sha256(message.encode())
    return int.from_bytes(hash_object.digest(), 'big')

In [28]:
def sign_elgamal(message, private_key, p, g):
    x = private_key
    H_m = hash_message(message)
    
    k = random.randint(2, p - 2)
    while gcd(k, p - 1) != 1:
        k = random.randint(2, p - 2)
    
    r = mod_exp(g, k, p)
    
    k_inverse = mod_inverse(k, p - 1)
    s = ((H_m - x * r) * k_inverse) % (p - 1)
    
    return r, s

In [29]:
def verify_elgamal_signature(message, signature, public_key):

    p, g, y = public_key

    r, s = signature

    # Bước 1: Băm thông điệp để lấy giá trị băm H_m
    H_m = hash_message(message)
    print(f"Bước 1: Giá trị băm của thông điệp H(m) = {H_m}")

    # Bước 2: Tính g^H_m mod p để có vế trái của phương trình
    left_side = mod_exp(g, H_m, p)
    print(f"Bước 2: Tính g^H(m) mod p = {left_side}")

    # Bước 3: Tính (y^r * r^s) mod p để có vế phải của phương trình
    y_r_mod_p = mod_exp(y, r, p)
    r_s_mod_p = mod_exp(r, s, p)
    right_side = (y_r_mod_p * r_s_mod_p) % p
    print(f"Bước 3: Tính y^r mod p = {y_r_mod_p}")
    print(f"Bước 3: Tính r^s mod p = {r_s_mod_p}")
    print(f"Bước 3: Tính (y^r * r^s) mod p = {right_side}")

    # Bước 4: So sánh vế trái và vế phải
    if left_side == right_side:
        print("Bước 4: Vế trái bằng vế phải. Chữ ký hợp lệ.")
    else:
        print("Bước 4: Vế trái không bằng vế phải. Chữ ký không hợp lệ.")

    # Kết quả cuối cùng
    return left_side == right_side


In [30]:
p = 344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303
print(f"p: {p}")
# g = 2
g = 13496366429637966183200866655908501844287542699618604731918713325753280286175497136016236614727621674635809206897089783790551319270201625116999741593674645525926986285973782042187324374542581942378861320998458518558357487645631881644112609323183361747147272964789761654258910528521592648905075324958458103809
print(f"g: {g}")
public_key, private_key = generate_elgamal_keys(p, g)
print(f"Khóa công khai: {public_key}")
print(f"Khóa bí mật: {private_key}")
message = "Hoang Kim Chi"
signature = sign_elgamal(message, private_key, p, g)
print(f"Chữ ký: {signature}")
is_valid = verify_elgamal_signature(message, signature, public_key)

p: 344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303
g: 13496366429637966183200866655908501844287542699618604731918713325753280286175497136016236614727621674635809206897089783790551319270201625116999741593674645525926986285973782042187324374542581942378861320998458518558357487645631881644112609323183361747147272964789761654258910528521592648905075324958458103809
Khóa công khai: (344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303, 13496366429637966183200866655908501844287542699